### Download the dataset

In [1]:
%env BASE_DIR=/home/jupyter/snippets

env: BASE_DIR=/home/jupyter/snippets


In [2]:
!gsutil cp gs://cs327e-open-access/fs_shopify.zip $BASE_DIR

Copying gs://cs327e-open-access/fs_shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [3]:
# Commented this out, cause already unzipped it
#!unzip $BASE_DIR/fs_shopify.zip

### Download Firestore code samples 

In [4]:
!gsutil cp gs://cs327e-open-access/fs_samples.zip $BASE_DIR

Copying gs://cs327e-open-access/fs_samples.zip...
/ [1 files][  4.5 KiB/  4.5 KiB]                                                
Operation completed over 1 objects/4.5 KiB.                                      


In [5]:
# Commented this out, cause already unzipped it
#!unzip $BASE_DIR/fs_samples.zip

### Create the Firestore collections and subcollections based on your data model and populate them with the Shopify records

In [6]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

# categories collection csv
df = pd.read_csv('fs_shopify/categories.csv', sep=',', header=0, lineterminator='\n')
categories_rows = df.values.tolist()

# app collection csv
df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

# needed this for the 3 loop to create a app subcollection under categories collection
df = pd.read_csv('fs_shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
apps_categories_rows = df.values.tolist()

# Categories Collection
for categories_row in categories_rows: 
    categories_record = {}
    categories_record['id'] = categories_row[0]
    categories_record['title'] = categories_row[1]
    
    categories_ref = db.collection('categories').document(categories_row[0])
    batch.set(categories_ref, categories_record)
    
    # app_categories link
    for apps_categories_row in apps_categories_rows:
        if categories_row[0] == apps_categories_row[1]:
            
            # App Subcollection
            for app_row in app_rows:
                if apps_categories_row[0] == app_row[0]:
                    app_record = {}
                    app_record['url'] = app_row[1]
                    app_record['title'] = app_row[2]
                    app_record['developer'] = app_row[3]
                    app_record['developer_link'] = app_row[4]
                    app_record['icon'] = app_row[5]
                    app_record['rating'] = app_row[6]
                    app_record['reviews_count'] = app_row[7]
                    
                    apps_ref = categories_ref.collection('apps').document(app_row[0])
                    
                    batch.set(apps_ref, app_record)
                    
    batch.commit()

In [7]:
import pandas as pd
from google.cloud import firestore
import uuid

db = firestore.Client()
batch = db.batch()

batch_size = 500

# reading the cvs files for app collection and their subcollections

# app collection csv
df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

# reviews subcollection csv
df = pd.read_csv('fs_shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
reviews_rows = df.values.tolist()

# key_benefits subcollection csv
df = pd.read_csv('fs_shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
key_benefits_rows = df.values.tolist()

# pricing_plans subcollection csv
df = pd.read_csv('fs_shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pricing_plans_rows = df.values.tolist()

# categories subcollection csv
df = pd.read_csv('fs_shopify/categories.csv', sep=',', header=0, lineterminator='\n')
categories_rows = df.values.tolist()

# needed this for the 3 loop to create a categories subcollection under app collection
df = pd.read_csv('fs_shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
apps_categories_rows = df.values.tolist()

# App Collection
for app_row in app_rows: 
    app_record = {}
    app_record['id'] = app_row[0]
    app_record['url'] = app_row[1]
    app_record['title'] = app_row[2]
    app_record['developer'] = app_row[3]
    app_record['developer_link'] = app_row[4]
    app_record['icon'] = app_row[5]
    app_record['rating'] = app_row[6]
    app_record['reviews_count'] = app_row[7]
    
    apps_ref = db.collection('apps').document(app_row[0])
    batch.set(apps_ref, app_record)
    
    # Review Subcollection
    for reviews_row in reviews_rows:
        if app_row[0] == reviews_row[0]:
            reviews_id = str(uuid.uuid4())
            reviews_record = {}
            reviews_record['author'] = reviews_row[1]
            reviews_record['rating'] = reviews_row[2]
            reviews_record['posted_at'] = reviews_row[3]
            
            reviews_ref = apps_ref.collection('reviews').document(reviews_id)
        
            batch.set(reviews_ref, reviews_record)
            
    # Key_Benefits Subcollection
    for key_benefits_row in key_benefits_rows:
        if app_row[0] == key_benefits_row[0]:
            key_benefits_id = str(uuid.uuid4())
            key_benefits_record = {}
            key_benefits_record['title'] = key_benefits_row[1]
            key_benefits_record['description'] = key_benefits_row[2]
            
            key_benefits_ref = apps_ref.collection('key_benefits').document(key_benefits_id)
        
            batch.set(key_benefits_ref, key_benefits_record)    
            
    # Pricing_plans Subcollection
    for pricing_plans_row in pricing_plans_rows:
        if app_row[0] == pricing_plans_row[1]: #second column is app_id
            pricing_plans_id = str(uuid.uuid4())
            pricing_plans_record = {}
            pricing_plans_record['author'] = pricing_plans_row[0]
            pricing_plans_record['rating'] = pricing_plans_row[2]
            pricing_plans_record['posted_at'] = pricing_plans_row[3]
            
            pricing_plans_ref = apps_ref.collection('pricing_plans').document(pricing_plans_id) #? or row 0,2,3
        
            batch.set(pricing_plans_ref, pricing_plans_record)
            
    # Categories Subcollection
    for apps_categories_row in apps_categories_rows:
        if app_row[0] == apps_categories_row[0]: # need app_categories to tie the category as a subcollection
            for categories_row in categories_rows:
                if apps_categories_row[1] == categories_row[0]:
                    categories_record = {}
                    categories_record['title'] = categories_row[1]
            
                    categories_ref = apps_ref.collection('categories').document(categories_row[0])
        
                    batch.set(categories_ref, categories_record)     
            
    # Commiting the batch in 500        
    if len(batch) >= batch_size:
        batch.commit()
        batch = db.batch() # Creating a new batch    
        
# Commit the remaining batch, if any
if batch is not None:
    batch.commit()

### Retrieve the number of documents in each collection and subcollection using count(). 
#### See code sample read_count.py.

In [9]:
# Counting the number of documents in Categories Collection
from google.cloud import firestore

db = firestore.Client()
categories_ref = db.collection('categories')
count_categories = categories_ref.count()
results = count_categories.get()
    
for result in results:
    print(f"Number of documents in the 'categories' collection: {result[0].value}")

Number of documents in the 'categories' collection: 12


In [10]:
# Counting the number of documents in Apps Collection
from google.cloud import firestore

db = firestore.Client()
apps_ref = db.collection('apps')
count_apps = apps_ref.count()
results = count_apps.get()
    
for result in results:
    print(f"Number of documents in the 'apps' collection: {result[0].value}")

Number of documents in the 'apps' collection: 3547


In [11]:
# Counting the number of documents in Apps Subcollection under Categories Collection
from google.cloud import firestore

db = firestore.Client()
categories_ref = db.collection('categories')

for categories in categories_ref.stream():
    categories_id = categories.id
    apps_ref = categories_ref.document(categories_id).collection('apps')
    apps_count = apps_ref.count().get()
    
    for result in apps_count:
        count_value = result[0].value
        print(f"Number of docs in 'apps' subcollection under category collection '{categories_id}': {count_value}")


Number of docs in 'apps' subcollection under category collection '26a72de0d02e0e4e5f615332d61a878e': 133
Number of docs in 'apps' subcollection under category collection '30a930262efca129caafcd586bc7e6fe': 83
Number of docs in 'apps' subcollection under category collection '30ea2315e910c5e5335de42d5e87dca5': 602
Number of docs in 'apps' subcollection under category collection '5eb4e29e50e3f178acc614236ed107f4': 957
Number of docs in 'apps' subcollection under category collection '64430ad2835be8ad60c59e7d44e4b0b1': 310
Number of docs in 'apps' subcollection under category collection '737ad50051083aa051d127a53b3ac0da': 190
Number of docs in 'apps' subcollection under category collection '9fe78acd078fc030de72c896adc93a2d': 251
Number of docs in 'apps' subcollection under category collection 'bd125b49ce6b0425d29e33cb8b49a496': 176
Number of docs in 'apps' subcollection under category collection 'c3f9db73c29bc33607778f9935c3c4dc': 1537
Number of docs in 'apps' subcollection under category c

In [12]:
# Counting the number of documents in Pricing_Plans Subcollection under Apps Collection

from google.cloud import firestore

db = firestore.Client()

apps_ref = db.collection('apps')

# Iterate through each app and count the documents in the "pricing_plans" subcollection
for app in apps_ref.stream():
    app_id = app.id #getting app id 
    pricing_plans_ref = apps_ref.document(app_id).collection('pricing_plans') #querying pricing plan for 1 particular app
    pricing_plans_count = pricing_plans_ref.count().get() #counting pricing plans
        
    for result in pricing_plans_count:
        count_value = result[0].value
        print(f"Number of docs in 'pricing_plans' subcollection under apps collection '{app_id}': {count_value}")

Number of docs in 'pricing_plans' subcollection under apps collection '00289a9f-9f12-45b1-963b-67e78403f7c7': 1
Number of docs in 'pricing_plans' subcollection under apps collection '00312cea-3595-4d65-a60d-6e8e5b7d6897': 2
Number of docs in 'pricing_plans' subcollection under apps collection '003b3dbd-1295-48cb-a3e8-8f3a0e8fc6df': 3
Number of docs in 'pricing_plans' subcollection under apps collection '004cc187-6def-4058-a7f3-78e6edfe2e3c': 1
Number of docs in 'pricing_plans' subcollection under apps collection '005d6c7a-9eb9-4b35-bdd4-040d383e86cf': 1
Number of docs in 'pricing_plans' subcollection under apps collection '0067a20e-e307-490e-acc5-8442d4bdca4b': 1
Number of docs in 'pricing_plans' subcollection under apps collection '008ce864-6073-4b61-b133-265ea6afa893': 1
Number of docs in 'pricing_plans' subcollection under apps collection '00951791-6206-4780-ba17-5e57520c277c': 1
Number of docs in 'pricing_plans' subcollection under apps collection '00c5d020-0877-4534-88b8-f05fd4cf5

In [13]:
# Counting the number of documents in Categories Subcollection under Apps Collection

from google.cloud import firestore

db = firestore.Client()

apps_ref = db.collection('apps')

# Iterate through each app and count the documents in the "categories" subcollection
for app in apps_ref.stream():
    app_id = app.id #getting app id 
    categories_ref = apps_ref.document(app_id).collection('categories') #querying pricing plan for 1 particular app
    categories_count = categories_ref.count().get() #counting pricing plans
        
    for result in categories_count:
        count_value = result[0].value
        print(f"Number of docs in 'categories' subcollection under apps collection '{app_id}': {count_value}")

Number of docs in 'categories' subcollection under apps collection '00289a9f-9f12-45b1-963b-67e78403f7c7': 2
Number of docs in 'categories' subcollection under apps collection '00312cea-3595-4d65-a60d-6e8e5b7d6897': 1
Number of docs in 'categories' subcollection under apps collection '003b3dbd-1295-48cb-a3e8-8f3a0e8fc6df': 2
Number of docs in 'categories' subcollection under apps collection '004cc187-6def-4058-a7f3-78e6edfe2e3c': 1
Number of docs in 'categories' subcollection under apps collection '005d6c7a-9eb9-4b35-bdd4-040d383e86cf': 2
Number of docs in 'categories' subcollection under apps collection '0067a20e-e307-490e-acc5-8442d4bdca4b': 2
Number of docs in 'categories' subcollection under apps collection '008ce864-6073-4b61-b133-265ea6afa893': 2
Number of docs in 'categories' subcollection under apps collection '00951791-6206-4780-ba17-5e57520c277c': 2
Number of docs in 'categories' subcollection under apps collection '00c5d020-0877-4534-88b8-f05fd4cf5fa6': 1
Number of docs in '

In [14]:
# Counting the number of documents in Key_Benefits Subcollection under Apps Collection

from google.cloud import firestore

db = firestore.Client()

apps_ref = db.collection('apps')

# Iterate through each app and count the documents in the "key_benefits" subcollection
for app in apps_ref.stream():
    app_id = app.id #getting app id 
    key_benefits_ref = apps_ref.document(app_id).collection('key_benefits') #querying pricing plan for 1 particular app
    key_benefits_count = key_benefits_ref.count().get()
        
    for result in key_benefits_count:
        count_value = result[0].value
        print(f"Number of docs in 'key_benefits' subcollection under apps collection '{app_id}': {count_value}")

Number of docs in 'key_benefits' subcollection under apps collection '00289a9f-9f12-45b1-963b-67e78403f7c7': 3
Number of docs in 'key_benefits' subcollection under apps collection '00312cea-3595-4d65-a60d-6e8e5b7d6897': 3
Number of docs in 'key_benefits' subcollection under apps collection '003b3dbd-1295-48cb-a3e8-8f3a0e8fc6df': 3
Number of docs in 'key_benefits' subcollection under apps collection '004cc187-6def-4058-a7f3-78e6edfe2e3c': 3
Number of docs in 'key_benefits' subcollection under apps collection '005d6c7a-9eb9-4b35-bdd4-040d383e86cf': 3
Number of docs in 'key_benefits' subcollection under apps collection '0067a20e-e307-490e-acc5-8442d4bdca4b': 3
Number of docs in 'key_benefits' subcollection under apps collection '008ce864-6073-4b61-b133-265ea6afa893': 3
Number of docs in 'key_benefits' subcollection under apps collection '00951791-6206-4780-ba17-5e57520c277c': 3
Number of docs in 'key_benefits' subcollection under apps collection '00c5d020-0877-4534-88b8-f05fd4cf5fa6': 3
N

In [15]:
# Counting the number of documents in Reviews Subcollection under Apps Collection

from google.cloud import firestore

db = firestore.Client()

apps_ref = db.collection('apps')

# Iterate through each app and count the documents in the "reviews" subcollection
for app in apps_ref.stream():
    app_id = app.id #getting app id 
    reviews_ref = apps_ref.document(app_id).collection('reviews') #querying pricing plan for 1 particular app
    reviews_count = reviews_ref.count().get()
        
    for result in reviews_count:
        count_value = result[0].value
        print(f"Number of docs in 'reviews' subcollection under apps collection '{app_id}': {count_value}")

Number of docs in 'reviews' subcollection under apps collection '00289a9f-9f12-45b1-963b-67e78403f7c7': 5
Number of docs in 'reviews' subcollection under apps collection '00312cea-3595-4d65-a60d-6e8e5b7d6897': 0
Number of docs in 'reviews' subcollection under apps collection '003b3dbd-1295-48cb-a3e8-8f3a0e8fc6df': 0
Number of docs in 'reviews' subcollection under apps collection '004cc187-6def-4058-a7f3-78e6edfe2e3c': 0
Number of docs in 'reviews' subcollection under apps collection '005d6c7a-9eb9-4b35-bdd4-040d383e86cf': 4
Number of docs in 'reviews' subcollection under apps collection '0067a20e-e307-490e-acc5-8442d4bdca4b': 107
Number of docs in 'reviews' subcollection under apps collection '008ce864-6073-4b61-b133-265ea6afa893': 0
Number of docs in 'reviews' subcollection under apps collection '00951791-6206-4780-ba17-5e57520c277c': 0
Number of docs in 'reviews' subcollection under apps collection '00c5d020-0877-4534-88b8-f05fd4cf5fa6': 0
Number of docs in 'reviews' subcollection un

### List the top 10 "Productivity" apps (whose categories.title = "Productivity") sorted by their rating in descending order. Return the id, title, developer, rating and reviews_count for those apps. Limit the results to the first 10 records. 

#### Note: this query refers to access pattern #1.

In [18]:
# Google is a god send, also lecture notes/slides, and recordings. 

from google.cloud import firestore

db = firestore.Client()

# First, query for the "Productivity" category
categories_ref = db.collection('categories')
query = categories_ref.where('title', '==', 'Productivity')

results = query.stream()

for result in results:
    category_id = result.id

    # Now that we have the "Productivity" category, query for the apps in this category
    apps_ref = categories_ref.document(category_id).collection('apps')
    apps_query = apps_ref.order_by('rating', direction=firestore.Query.DESCENDING).limit(10)

    app_results = apps_query.stream()

    for app_result in app_results:
        app_data = app_result.to_dict()
        app_id = app_result.id

        print(f'App ID: {app_id}')
        print(f'Title: {app_data["title"]}')
        print(f'Developer: {app_data["developer"]}')
        print(f'Rating: {app_data["rating"]}')
        print(f'Reviews Count: {app_data["reviews_count"]}')
        print('---')

/opt/conda/lib/python3.10/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


App ID: fe5ae45c-379c-42bf-be7d-b5f6e15fc13c
Title: Customer Tags
Developer: Union Works Apps
Rating: 5.0
Reviews Count: 21
---
App ID: f99bb1e3-f326-4f10-8901-491652e9809b
Title: Order Tagger
Developer: Union Works Apps
Rating: 5.0
Reviews Count: 68
---
App ID: f864e3bd-da0e-41dc-be65-984325331475
Title: SilkRoad ‑ Facebook Auto Ads
Developer: SilkRoad
Rating: 5.0
Reviews Count: 2
---
App ID: f5344e64-9cda-4d97-b198-2aaeb5170518
Title: Xporter Data Export Tool
Developer: Modd Apps Inc.
Rating: 5.0
Reviews Count: 223
---
App ID: ed77a32d-0fa3-458b-b639-e01ea7b78ec0
Title: FraudBlock Fraud Prevention
Developer: ShopFox
Rating: 5.0
Reviews Count: 6
---
App ID: e0e231d6-4988-4a8c-ad84-cf18d0f38738
Title: Automation Fox
Developer: Automation Fox
Rating: 5.0
Reviews Count: 1
---
App ID: d75fa395-3a4d-41c3-88d1-2f802c1e2411
Title: AirPower
Developer: BaseGenius
Rating: 5.0
Reviews Count: 1
---
App ID: d0be03ff-f74d-4fb5-8d2c-79541a1aea1b
Title: Excelify
Developer: Excelify.io
Rating: 5.0
Rev

### List the 10 apps with the highest number of reviews (based on apps.review_count). Return the id, title, developer, rating and reviews_count for those apps. Order the results by reviews_count in descending order.
#### Note: this is query refers to access pattern #2.

In [19]:
# Access Pattern 2: Get apps with highest review_count

from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter

db = firestore.Client()

apps_ref = db.collection('apps')

query = apps_ref.order_by('reviews_count', direction = firestore.Query.DESCENDING).limit(10)

results = query.stream()

for result in results:
    app_data = result.to_dict()
    app_id = result.id
    print(f'App ID: {app_id}')
    print(f'Title: {app_data["title"]}')
    print(f'Developer: {app_data["developer"]}')
    print(f'Rating: {app_data["rating"]}')
    print(f'Reviews Count: {app_data["reviews_count"]}')
    print('---')

App ID: d9f142ee-b141-4dc4-9353-173db61d2eb0
Title: Privy ‑ Exit Pop Ups & Email
Developer: Privy
Rating: 4.7
Reviews Count: 23078
---
App ID: 78ea0810-c008-4a4e-a82f-de0c790e3286
Title: Free Shipping Bar
Developer: Hextom
Rating: 4.9
Reviews Count: 8737
---
App ID: b88488b0-9912-44d3-b736-224c36f09d95
Title: Sales Pop ‑ Popup Notification
Developer: CartKit
Rating: 4.8
Reviews Count: 6905
---
App ID: e528a60e-94f8-4e92-80e2-5bc6013b8283
Title: BEST Currency Converter
Developer: Grizzly Apps
Rating: 4.8
Reviews Count: 5986
---
App ID: be2640c4-01b5-4d52-9f68-cae8c0734d0d
Title: Recart FB Messenger Marketing
Developer: Recart
Rating: 4.8
Reviews Count: 5596
---
App ID: 70bff9e0-4316-4cc6-84ce-92fcd1bc6925
Title: EU Cookie Bar ‑ Cookie GDPR
Developer: Booster Apps
Rating: 4.7
Reviews Count: 5259
---
App ID: 171816e2-27d4-4552-a65e-ab44a312fe04
Title: Sales Pop Master ‑ Countdown
Developer: Autoketing
Rating: 4.8
Reviews Count: 4931
---
App ID: 9025eff0-d714-4df1-930f-43f5582979ad
Title: 